In [1]:
import Pkg
Pkg.activate(@__DIR__)
# Pkg.upgrade_manifest()
# Pkg.update()
Pkg.resolve()
Pkg.instantiate()

import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
import MuJoCo
using LinearAlgebra
import Plots
using Random
using JLD2
using Test
using StaticArrays
using Printf
using Distributions
using MathOptInterface

  Activating project at `~/daniel/ocrl-piano/single_finger_free_trajectories`
  No Changes to `~/daniel/ocrl-piano/single_finger_free_trajectories/Project.toml`
  No Changes to `~/daniel/ocrl-piano/single_finger_free_trajectories/Manifest.toml`


In [2]:
Threads.nthreads()
using MuJoCo
install_visualiser()

   Resolving package versions...
  No Changes to `~/daniel/ocrl-piano/single_finger_free_trajectories/Project.toml`
  No Changes to `~/daniel/ocrl-piano/single_finger_free_trajectories/Manifest.toml`


### Load Model and Info

In [3]:
# model = load_model("../mujoco_menagerie/shadow_hand/scene_right_piano_hand.xml")
println(pwd())
model = load_model("models/scene_right_piano_hand.xml")
# model = load_model("scene_right_piano_hand.xml")
data = init_data(model)

nx = 2*model.nv
nu = model.nu

println("States: ", nx)
println("Controls: ", nu)
ϵ = 1e-6

init_visualiser()
function random_controller!(m::Model, d::Data)
    nu = m.nu
    d.ctrl .= 2*rand(nu) .- 1
    return nothing
end

println("Initial joint positions:", data.qpos)
println("Initial joint velocities:", data.qvel)

mj_resetData(model, data)

## Random Controller
# for t in 1:100
#     random_controller!(model, data)
#     step!(model, data)
# end
# println("New joint positions:", data.qpos)

# visualise!(model, data, controller=random_controller!)
print(data)

/home/workstation4/daniel/ocrl-piano/single_finger_free_trajectories
States: 54
Controls: 23


┌ Warning: The visualiser for MuJoCo.jl is designed to run multi-threaded, but the current Julia session was started with only one thread. Degraded performance will occur. To enable multi-threading, set JULIA_NUM_THREADS to a value greater than 1 before starting Julia.
└ @ VisualiserExt /home/workstation4/.julia/packages/MuJoCo/9JERS/ext/VisualiserExt/VisualiserExt.jl:76


Initial joint positions:[0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0;;]
Initial joint velocities:[0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0;;]
MuJoCo Data

### Keyframe Visualization

In [4]:
resetkey!(model, data, 3)
# visualise!(model, data)
data.qpos

27×1 transpose(::UnsafeArrays.UnsafeArray{Float64, 2}) with eltype Float64:
 -4.80347e-5
  0.309999
  8.98153e-5
 -1.02167e-11
  0.275076
  2.8028e-12
  1.15348e-10
  6.50327e-11
 -5.03929e-12
  2.80119e-12
  ⋮
 -2.80628e-12
  1.15023e-10
  6.49795e-11
 -5.03721e-12
 -5.36153e-14
  1.45269e-10
  9.38137e-11
 -1.74212e-11
 -5.83979e-12

### Dynamics

In [5]:
function wrapped_mj_step(model, data, xk, uk)
    """
    wrapped_mj_step(model, data, xk, uk)

    Computes the next state x_{k+1} given the current state x_k and control uk
    """
    # given the current model and data. set the state and control to the model and perform a forward step
    if typeof(xk) == Vector{Float64}
    
        data.qpos .= xk[1:model.nq]
        data.qvel .= xk[(model.nq + 1):end]
        data.ctrl .= uk

    else
        # if using diff types, we need to convert the dual numbers to floats
        # uk = ForwardDiff.value.(uk)
        # xk = ForwardDiff.value.(x)
        # # set control
        # data.ctrl[:] .= converted_uk

        # # set state
        # data.qpos .= converted_x[1:model.nq]
        # data.qvel .= converted_x[(model.nq + 1):end]

        xk = ForwardDiff.value(xk)
        uk = ForwardDiff.value(uk)

        data.qpos .= xk[1:model.nq]
        data.qvel .= xk[(model.nq + 1):end]
        data.ctrl .= uk
    end
    
    # take discrete dynamics step 
    step!(model, data) 

    # return updated state k + 1
    zkp1 = zeros(model.nq + model.nv + model.na) 
    zkp1 .= get_physics_state(model, data)


    # finger_coordinates = data.geom_xpos[finger_geom_indices,:]
    
    return zkp1
end

function robohand_cost(params::NamedTuple, Z::Vector)::Real
    # TODO: implement cost function
    idx, N, xg = params.idx, params.N, params.xg
    model = params.model
    data = params.data
    Q, R, Qf = params.Q, params.R, params.Qf

    # stage cost
    cost = 0.0
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]

        # @show xi-xg
        # cost += 0.5*(xi[94:96]-xg[94:96])'*Q*(xi[94:96]-xg[94:96])+0.5*ui'*R*ui
        cost += 0.5*(xi-xg)'*Q*(xi-xg)+0.5*ui'*R*ui
    end

    # terminal cost 
    xf = Z[idx.x[N]]
    # cost += 0.5*(xf[94:96]-xg[94:96])'*Qf*(xf[94:96]-xg[94:96])
    cost += 0.5*(xf-xg)'*Qf*(xf-xg)
    # @show typeof(cost)
    return cost
end

function robohand_temporal_pose_cost(params::NamedTuple, Z::Vector)::Real
    idx, N, xg= params.idx, params.N , params.xg
    model = params.model
    data = params.data
    Q, R, Qf = params.Q, params.R, params.Qf

    cost = 0.0
    
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]

        if i < N/2
            resetkey!(model, data, 2)
        else
            resetkey!(model, data, 3)
        end
        goal_pose = vec([data.qpos; data.qvel])

        # @show typeof(goal_pose)
        # @show goal_pose
        # @show typeof(xi-goal_pose)
        # @show xi-goal_pose

        # println()
        # @show xg
        # @show typeof(xg)
        # @show typeof(xi-xg)
        # @show xi-xg

        cost += 0.5*(xi-goal_pose)'*Q*(xi-goal_pose)+0.5*ui'*R*ui
    end
    # @show typeof(cost)
    return cost
end

function robohand_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    model = params.model
    data = params.data
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)
    
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]] 
        xip1 = Z[idx.x[i+1]]
        
        # data.qpos[94:96] = xi[1:3]
        # data.qvel[94:96] = xi[4:6]

        xip1_mujoco = wrapped_mj_step(model, data, xi, ui)

        c[idx.c[i]] = xip1_mujoco - xip1
    end
    # println(typeof(c))
    return c 
end


function robohand_equality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement equality constraints
    # return zeros(eltype(Z), 0)
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    model = params.model
    data = params.data

    con_1 = Z[idx.x[1]] - xic
    con_2 = Z[idx.x[N]] - xg

    # return zeros(eltype(Z), 0)
    # return [con_1; con_2]
    # return [con_1; con_2; robohand_dynamics_constraints(params, Z)]
    return [con_1; robohand_dynamics_constraints(params, Z)]


end

function robohand_inequality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement inequality constraints
    # println(Z)
    return zeros(eltype(Z), 0)
    # return inequality_constraints
end

function create_idx(nx,nu,N)
    # This function creates some useful indexing tools for Z 
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end


create_idx (generic function with 1 method)

### Test Equality Constraints

In [6]:
# model = load_model("models/scene_right_piano_hand.xml")
# data = init_data(model)

# # reset the model and data
# reset!(model, data)

# nx = 2*model.nv
# nu = model.nu

# # initiate time and time steps
# dt = model.opt.timestep
# tf = 5.0
# t_vec = 0:dt:tf
# N = length(t_vec)

# # LQR cost
# Q = diagm(ones(model.nq + model.nv))
# R = 100*diagm(ones(model.nu))
# Qf = Q

# # initial and goal states
# xic = vec(vcat(copy(data.qpos), copy(data.qvel)))
# println("Initial state: ", xic)

# # xg = vec(vcat(copy(data.qpos), copy(data.qvel)))
# xg = vec(vcat(copy(data.qpos), copy(data.qvel)))
# # xg[1] = deg2rad(2)
# println("Goal state: ", xg)

# # indexing 
# idx = create_idx(model.nq + model.nv, model.nu, N)
# idx = create_idx(nx, nu, N)
# params = (Q = Q, R = R, Qf = Qf, xic = xic, xg = xg, dt = dt, N = N, idx = idx, model=model, data=data)

# z0 = zeros(idx.nz)
# q0_array = -0.001:0.0001:0.001

# # costs = zeros(length(q0_array))
# # constraint_cost = zeros(length(q0_array))


# costs = zeros(idx.nz, length(q0_array)) 
# constraint_cost = zeros(idx.nz, length(q0_array))
# using Plots

# p1 = plot()
# p2 = plot()
# for j in 1:idx.nz
#     for i in eachindex(q0_array)
#         z0[j] = q0_array[i]
#         # z0[idx.x[i]] .= q0_array[i]
#         costs[j, i] = robohand_cost(params, z0)
#         constraint_cost[j, i] = norm(robohand_dynamics_constraints(params, z0), 2)
#     end
#     z0 = zeros(idx.nz)

#     # p1 = plot!(q0_array, costs[j, :], label="$j", xlabel="q0", ylabel="Cost", legend=false)
#     # p2 = plot!(q0_array, constraint_cost[j, :], label="$j", xlabel="q0", ylabel="Constraint cost")
#     p2 = plot!(q0_array, constraint_cost[j, :], xlabel="q0", ylabel="Constraint cost", legend=false)

# end

# # display(p1)
# display(p2)

# # p1 = plot(q0_array, costs, label="Cost vs q0", xlabel="q0", ylabel="Cost")
# # p2 = plot(q0_array, constraint_cost, label="Constraint cost vs q0", xlabel="q0", ylabel="Constraint cost")

# # println()
# # println("Constraint Cost: ", constraint_cost)

# # plot(p1, p2, layout=(2, 1), size=(600, 400))

### DIRCOL

In [7]:
include(joinpath(@__DIR__, "utils","fmincon.jl"))

fmincon (generic function with 1 method)

In [8]:
function solve_hand_ddp(;verbose=true)
    # instantiate model and data
    model = load_model("models/scene_right_piano_hand.xml")
    data = init_data(model)

    # reset the model and data
    reset!(model, data)

    nx = 2*model.nv
    nu = model.nu

    # initiate time and time steps
    model.opt.timestep = 0.1
    dt = model.opt.timestep
    tf = 2.0
    t_vec = 0:dt:tf
    N = length(t_vec)

    # LQR cost

    q_diag = ones(model.nq + model.nv)
    q_diag[1:3] .= 50 # prioritize the position of the hand 
    # Q = diagm(ones(model.nq + model.nv))
    Q = diagm(q_diag)

    
    R = 0.05*diagm(ones(model.nu))
    Qf = Q

    # initial and goal states
    xic = vec(vcat(copy(data.qpos), copy(data.qvel)))
 
    println("Initial state: ", xic)

    # xg = vec(vcat(copy(data.qpos), copy(data.qvel)))
    xg = vec(vcat(copy(data.qpos), copy(data.qvel)))
    xg[2] = 0.1
    xg[5] = deg2rad(5)
    xg[21] = deg2rad(15)

    println("Goal state: ", xg)

    # indexing 
    idx = create_idx(model.nq + model.nv, model.nu, N)
    idx = create_idx(nx, nu, N)
    params = (Q = Q, R = R, Qf = Qf, xic = xic, xg = xg, dt = dt, N = N, idx = idx, model=model, data=data)


    # primal bounds
    # x_l = vcat(model.jnt_range[:,1], -Inf*ones(model.nv), model.actuator_ctrlrange[:,1]) # combine joint pos, vel, and control limits
    # x_u = vcat(model.jnt_range[:,2], Inf*ones(model.nv), model.actuator_ctrlrange[:,2])
    x_l = -Inf*ones(idx.nz)
    x_u = Inf*ones(idx.nz)

    # x_l = zeros(idx.nz)
    # x_u = zeros(idx.nz)
    # for i = 1:N
    #     # x_l[idx.x[i]] = vcat(model.jnt_range[:,1], -Inf*ones(model.nv))
    #     # x_u[idx.x[i]] = vcat(model.jnt_range[:,2], Inf*ones(model.nv))
    #     x_l[idx.x[i]] = vcat(model.jnt_range[:,1], -deg2rad(10)*ones(model.nv))
    #     x_u[idx.x[i]] = vcat(model.jnt_range[:,2], deg2rad(10)*ones(model.nv))



    #     if i < N
    #         x_l[idx.u[i]] = model.actuator_ctrlrange[:,1]
    #         x_u[idx.u[i]] = model.actuator_ctrlrange[:,2]
    #     end
    # end

    # inequality constraints
    # c_l = -1 * Inf * ones(3*(idx.N-1))
    # c_u = Inf * ones(3*(idx.N-1))
    c_l = zeros(0)
    c_u = zeros(0)

    # initial guess 
    z0 = 0.001*randn(idx.nz)
    # z0[idx.x[1]] = xic
    # z0 = zeros(idx.nz)


    # diff type
    # diff_type = :auto 
    diff_type = :finite
    
    
    Z = fmincon(robohand_temporal_pose_cost, robohand_equality_constraints, robohand_inequality_constraints,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-6, c_tol = 1e-4, max_iters = 10, verbose = verbose)
    
    # pull the X and U solutions out of Z 
    X = [Z[idx.x[i]] for i = 1:N]
    U = [Z[idx.u[i]] for i = 1:(N-1)]
    return X, U, t_vec, params
    # return
end

solve_hand_ddp (generic function with 1 method)

In [9]:
X, U, t_vec, params = solve_hand_ddp(verbose=true)

Initial state: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Goal state: [0.0, 0.1, 0.0, 0.0, 0.08726646259971647, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2617993877991494, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :finite (FiniteDiff.jl)---
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------

****************************

([[-5.674061051916334e-28, 0.0, 0.0, 0.0, 0.0, 0.0, -2.3101144496936017e-29, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.248442099299652e-28, -5.490826271448009e-29, -3.0076862755506585e-29, 2.419748011777845e-29], [0.003891507757016083, -0.01998655670699664, 0.0021401573777437644, 0.009173667280594785, 0.006311851881674267, -0.006008735875666353, 0.0011995383454060583, 0.0398249723005773, 0.040517754067783074, -0.0016792123433666396  …  0.020144082534963904, 0.014855535832774713, -0.010040492738593091, -0.0008558449382528521, -0.0007698919384106195, 0.0007309172278167026, -0.002171336493545264, 0.0006488348663750196, -0.003359418559545628, -4.040046931469204e-6], [0.00996128986238567, -0.050057174649456575, 0.005699381511197967, 0.01825477587304663, 0.014403930616197165, -0.009800672787679057, 0.0015072632895534737, 0.07379324063318633, 0.07473090734922422, -0.0038523472611247736  …  -0.0025751556507535023, 0.02387902078964798, -0.0017768199802871282, 0.00011255490390541016, -0.

In [10]:
# instantiate model and data
model = load_model("models/scene_right_piano_hand.xml")
data = init_data(model)

# reset the model and data
reset!(model, data)
nx = model.nq + model.nv + model.na # State vector dimension
N = length(t_vec)
states = zeros(nx, N)
ctrl_states = zeros(model.nu, N)

states[:,1] = get_physics_state(model, data)


for t in 1:N
    # reset!(model, data)
    data.qpos .= X[t][1:model.nq]
    data.qvel .= X[t][model.nq+1:model.nq+model.nv]
    states[:,t] = get_physics_state(model, data)
end

visualise!(model, data, trajectories = states)

 __  __            _        _____       _ _ 
|  \/  |          | |      / ____|     (_) |
| \  / |_   _     | | ___ | |     ___   _| |
| |\/| | | | |_   | |/ _ \| |    / _ \ | | |
| |  | | |_| | |__| | (_) | |___| (_) || | |
|_|  |_|\__,_|\____/ \___/ \_____\___(_) |_|
                                      _/ |  
                                     |__/   

Press "F1" to show the help message.


In [11]:
mj_name2id(model, 1, "rh_ffdistal")

7

### MuJoCo Integrator Test
Compare manually calling wrapped_mj_step versus the actual simulator trajectories on the different MuJoCo integrators

In [12]:
# # instantiate model and data
# model = load_model("models/scene_right_piano_hand.xml")
# data = init_data(model)

# # reset the model and data
# reset!(model, data)

# nx = model.nq + model.nv
# nu = model.nu

# # timesteps
# dt = model.opt.timestep
# tf = 3.0
# t_vec = 0:dt:tf
# N = length(t_vec)

# # indexing 
# idx = create_idx(model.nq + model.nv, model.nu, N)
# idx = create_idx(nx, nu, N)

# # xkp1 = wrapped_mj_step(model, data, z0[idx.x[1]], z0[idx.u[1]])

# x_ic = vec(vcat(copy(data.qpos), copy(data.qvel)))
# z = zeros(nx, N)
# z[:,1] = xic

# for k in 1:(N-1)
#     # xk = z[idx.x[k]]
#     # uk = zeros(nu)
#     xk = z[:,k]
#     uk = zeros(nu)
#     # uk[1] = 0.2
#     # uk[10] = 0.2
#     # uk[15] = 0.3

#     zkp1 = wrapped_mj_step(model, data, xk, uk)
#     xkp1 = zkp1[1:nx]

#     # z[idx.x[k+1]] = xkp1
#     z[:,k+1] = xkp1
# end

# reset!(model, data)
# visualise!(model, data, trajectories=[z])
